In [1]:
import mysql.connector

In [2]:
mydb = mysql.connector.connect(
    host="localhost",        # or your MySQL server address
    user="root",    # e.g., "root"
    password="Hlw*123#", # your MySQL password
  
)
mycursor=mydb.cursor()
mycursor.execute("create database if not exists sales_mang")


In [3]:
mycursor.execute("use sales_mang")

In [4]:
mycursor.execute("create table if not exists login(username varchar(25) not null, password varchar(25))")


In [5]:
mycursor.execute("create table if not exists purchase(odate date not null, name varchar(25) not null, pcode int not null,amount int not null)")

In [6]:
mycursor.execute("create table if not exists stock(pcode int not null, pname varchar(25) not null, quantity int not null, price int not null)")

In [7]:
mydb.commit()

In [8]:
z=0
mycursor.execute("select*from login")
for i in mycursor:
  z+=1
if(z==0):
    mycursor.execute("insert into login values('username','ng')")
    mydb.commit()

In [9]:
def update_price(product_code,new_price):
    try:
        sql="update stock set price= %s where pcode=%s "
        values=(new_price,product_code)
        mycursor.execute(sql,values)
        mydb.commit()
        
        if mycursor.rowcount>0:
            print(f"Price updated successfully for the product code: {product_code}")
        else:
            print("No product wuth the given product code.")
    
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [10]:
def delete_product(pcode):
    try:
        sql="delete from stock where pcode=%s"
        values=(pcode,)
        mycursor.execute(sql,values)
        mydb.commit()
        
        if mycursor.rowcount >0:
            print(f"Product with code {pcode} is successfully deleted.")
        else:
            print("No such product is found.")
            
    except mysql.connector.Error as err:
        print(f"Error: {err}")

In [11]:
def insert_item(odate, pname, pcode, quantity):
    try:
      
        mycursor.execute("select * from stock where pcode=%s", (pcode,))
        result = mycursor.fetchone()  
        
        if result:
            t_code, t_name, t_quant, t_price = result
            if quantity > t_quant:
                print("Not enough stock available.")
                return
            
            
            amount = quantity * t_price
            
            sql = "insert into purchase(odate, pcode, name, quantity, amount) values (%s, %s, %s, %s, %s)"
            values = (odate, pcode, pname, quantity, amount)
            mycursor.execute(sql, values)
            
            
            mydb.commit()
            print(f"Added to bucket successfully. {quantity} units of {pname} .")
        else:
            print("Product not found in stock.")
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")



In [12]:
def display_items():
    try:
            mycursor.execute("select* from stock")
            result=mycursor.fetchall()
            print(f"{'product_code':<12} | {'product_name':<12} | {'price':<12} | {'quantity':<12}")
            print("-"*50)
            for i in result:
                t_code,t_name,t_price,t_qunt=i
                print(f"{t_code:<12} | {t_name:<12} | {t_price:<12} | {t_qunt:<12}")

    except mysql.connector.Error as err:
     print(f"Error: {err}")

In [13]:
while True:
    print("""1. Admin
    2. Customer
    3. Exit""")
    ch = int(input("Enter your choice: "))

    if ch == 1:
        pass_input = input("Enter your password: ")
        mycursor.execute("SELECT username, password FROM login")
        result = mycursor.fetchall()
        for username, password in result:
            if pass_input == password:
                print(f"Welcome, {username}")
                loop2 = 'y'
                
                while loop2.lower() == 'y': 
                    print("""1. Add new Item
                             2. Update price
                             3. Delete Item
                             4. Display all items
                             5. Change the password
                             6. Log out""")

                    kh = int(input("Enter your choice: "))

                    if kh == 1:  
                        loop = 'y'
                        while loop.lower() == 'y': 
                            pcode = int(input("Enter product code: "))
                            pname = input("Enter product name: ")
                            quantity = input("Enter product quantity: ")
                            price = int(input("Enter product price: "))
                            mycursor.execute("INSERT INTO stock(pcode, pname, quantity, price) VALUES(%s, %s, %s, %s)", (pcode, pname, quantity, price))
                            mydb.commit()
                            print("Record added successfully")
                            loop = input("Do you want to continue adding more values? (y/n): ")

                    elif kh==2:
                        loop='y'
                        while loop.lower()=='y':
                            pcode=int (input("Enter the pcode: "))
                            new_price=int(input("Enter the new price: "))
                            update_price(pcode,new_price)
                            loop=input("do u wanna update anymore?: ")
                            
                    elif kh==3:
                            loop='y'
                            while loop.lower()=='y' :
                                pcode=int(input("Enter the pcode u want to delete: "))
                                delete_product(pcode)
                                loop=input("do have more product to delete y/n: ")
                                
                    elif kh==4:
                         display_items()
                            
                    elif kh==5:
                            old_password=input("Enter your old password: ")
                            mycursor.execute("select * from login")
                            result=mycursor.fetchall()
                            for i in result:
                                username,password=i
                            if old_password==password:
                                new_pass=input("Enter ur new password: ")
                                mycursor.execute("update login set password=%s where username=%s",(new_pass,username))
                                mydb.commit()
                                print("password changed successfully.")
                            else:
                                print("Incorrect password.")
                            
                    elif kh == 6: 
                        print("Logging out...")
                        break 
                   
                    loop2 = input("Do you want to continue managing stocks? (y/n): ")
                break 
                
        else:
            print("Wrong password")
    
    elif ch == 2:
        print("Customer section......")
        loop2='y'
        while loop2.lower()=='y':
            print("""\n\n\n 1. Item bucket \n2.Payment \n3.View available items \n4.back""")
            ch=int(input("Enter ur choice: "))
            if(ch==1):
                loop='y'
                while loop.lower()=='y':
                    name=input("Enter the name: ")
                    pcode=int(input("Enter the product code: "))
                    quantity=int(input("Enter the quantity: "))
                    odate=input("Enter the date")
                    insert_item(odate,name,pcode,quantity)
                    loop=input("Do wanna insert more items: ")
                    
            elif ch == 2:
                try:
                    mycursor.execute("SELECT SUM(amount) AS total_amount FROM purchase")
                    total_amount = mycursor.fetchone()[0]  

                   
                    if total_amount is not None:
                        print(f"Amount to be paid: {total_amount}")
                    else:
                        print("No items found for payment.")

                except mysql.connector.Error as err:
                    print(f"Error: {err}")
                    
            elif ch==3:
                display_items()
                



            elif ch==4:
                print("ok!!")
                break

            loop2=input("do u wanna more editing: ")
            
    elif ch == 3:
        print("Exiting.....")
        break
    
    else:
        print("Invalid choice, please try again.")


1. Admin
    2. Customer
    3. Exit
Enter your choice: 2
Customer section......



 1. Item bucket 
2.Payment 
3.View available items 
4.back
Enter ur choice: 1
Enter the name: milk
Enter the product code: 3
Enter the quantity: 3
Enter the date3/3/3
Product not found in stock.
Do wanna insert more items: n
do u wanna more editing: y



 1. Item bucket 
2.Payment 
3.View available items 
4.back
Enter ur choice: 3
product_code | product_name | price        | quantity    
--------------------------------------------------
2            | choco        | 81           | 5           
4            | oil          | 7            | 5           
5            | cake         | 5            | 50          
do u wanna more editing: n
1. Admin
    2. Customer
    3. Exit
Enter your choice: 2
Customer section......



 1. Item bucket 
2.Payment 
3.View available items 
4.back
Enter ur choice: 2
Amount to be paid: 55.0
do u wanna more editing: n
1. Admin
    2. Customer
    3. Exit
Enter your choice: 3
Ex